In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import codecs
import os
import pandas as pd
import numpy as np
import regex
from collections import Counter
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import re 

In [54]:
bible_ver_dic = {}

new_bib = pd.read_csv("./data/bible/t_web.csv", index_col = [0])
old_bib = pd.read_csv("./data/bible/t_kjv.csv", index_col = [0])
literal_bib = pd.read_csv("./data/bible/t_ylt.csv", index_col = [0])

new_bib.t = new_bib.t.str.lower()
old_bib.t = old_bib.t.str.lower()
literal_bib.t = literal_bib.t.str.lower()


def text_cleaner(text):
    newString = re.sub("[\{\[].*?[\}\]]", "", text)
    newString = re.sub('"','', newString)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString

# remove notes from the bible text
new_bib.t = new_bib.t.apply(text_cleaner)
old_bib.t = old_bib.t.apply(text_cleaner)
literal_bib.t = literal_bib.t.apply(text_cleaner)

bibles = new_bib.copy()
bibles['new_bib'] = new_bib.t
bibles['old_bib'] = old_bib.t
bibles['literal_bib'] = literal_bib.t
bibles = bibles.drop(columns='t')

np.where(bibles.applymap(lambda x: x == ''))[0]


empty_rows = bibles[bibles['new_bib'] == ''].index
bibles = bibles.drop(empty_rows)
np.where(bibles.applymap(lambda x: x == ''))[0]



bibles = bibles.sample(frac=1)
print(bibles.shape)
bibles.head()
bibles.to_csv('./data/preprocessed/bibles_suffled.csv')

train = round(31096 * 0.7)
val = round(31096 * 0.2)
test = round(31096 * 0.1)
print(train, val, test, 'sum:',train+test+val)

(31095, 6)
21767 6219 3110 sum: 31096


In [55]:
train = round(31096 * 0.7)
val = round(31096 * 0.2)
test = round(31096 * 0.1)
print(train, val, test, 'sum:',train+test+val)

21767 6219 3110 sum: 31096


In [56]:
train_df = bibles[0:train].copy()
val_df = bibles[train:val+train].copy()
test_df = bibles[val+train:test+val+train].copy()


In [57]:
train_df['trg'] = train_df['literal_bib']
train_df['src'] = train_df['new_bib']
train_df = train_df.drop(columns = ['b', 'c', 'v', 'new_bib', 'old_bib', 'literal_bib'])

val_df['trg'] = val_df['literal_bib']
val_df['src'] = val_df['new_bib']
val_df = val_df.drop(columns = ['b', 'c', 'v', 'new_bib', 'old_bib', 'literal_bib'])

test_df['trg'] = test_df['literal_bib']
test_df['src'] = test_df['new_bib']
test_df = test_df.drop(columns = ['b', 'c', 'v', 'new_bib', 'old_bib', 'literal_bib'])

In [60]:
train_df.to_csv('./data/preprocessed/train_df.csv', header=None)
val_df.to_csv('./data/preprocessed/val_df.csv', header=None)
test_df.to_csv('./data/preprocessed/test_df.csv', header=None)